# Open University Data Analysis Workshop Project

Served by:
Itay Cohen - 209146158
Alon Gilda - 209146224

## Project Goal

The goal of this project is to give us users a better way to plan their trips using public transportation.
 
Often times when we plan a trip using public transportation and we count on the timings specified by the public transportation companies, we find ourselves waiting for the bus for a long time, or even worse, missing the bus and having to wait for the next one.

This project aims to solve this problem by using real-time data to predict the arrival time of the bus to the station, and to give the user a better estimation of the arrival time of the bus to the station.

## Motivation

We both live outside of Tel Aviv and have a long commute to work.

We prefer to relay on public transportation rather than driving to work.

Current applications and methods in Israel are far from accurate. We find ourselves not trusting the applications and prefer to use our own estimations based on our personal experience.

We believe that we can use the data that is available to us to create a better estimation of the arrival time of the bus to the station.




# Data introduction

There is a lot of data around public transportation methods, most of it relies on government source.

These sources listed below, are what we found best to describe the data we need to create our model. 

### GTFS

GTFS is a standard format for public transportation data. It is used by many public transportation companies around the world.

In Israel, GTFS data is being published in government websites, as long with some other private archives.
 
You can find here a detailed example of what GTFS data looks like:

[https://transitfeeds.com/p/ministry-of-transport-and-road-safety/820/latest/routes](https://transitfeeds.com/p/ministry-of-transport-and-road-safety/820/latest/routes)

*GTFS Data only contains planned data, and does not contain real-time data.*

### SIRI (Real time data)

SIRI Data is the Israeli government name for it's real-time bus API.

It's being published by a specific API upon request, and gives the opportunity to log real time data.

[https://www.gov.il/he/departments/general/real_time_information_siri](https://www.gov.il/he/departments/general/real_time_information_siri)

### Hasadna - Open Bus

Hasadna is a non-profit organization, based on volunteers, that collects data from the Israeli government and publishes it to the public.


Open Bus is a project by Hasadna that collects data from the SIRI API and publishes it to the public.

They log the data into servers and allow querying it via an API called Stride.

Hasadna Site : [www.hasadna.org.il](www.hasadna.org.il)


### Install dependencies

This notebook requires two dependencies which can be installed with the following command `pip install pandas open-bus-stride-client`.


In [ ]:
import pandas as pd
import math
import stride
import plotly.express as px
import seaborn as sns

# Routes to investigate

For the purpose of this project initial show, we've decided to showcase the data and the workflow for only 2 lines.

In the real work we will aim to use more lines and more data, depends on the computational limitations we will have.

We chose two specific lines and queried them offline via the transitfeeds website:

* 970 Haifa <> Bnei Brak -- Egged -- line ref: 19713, operator ref: 3
* 202 Pardes hana <> Hof Carmel -- Egged -- line ref: 19717, operator ref: 3

# Load GTFS Data

In [ ]:
# Open bus stride API
# https://open-bus-stride-api.hasadna.org.il/docs#/user%20cases/list__stop_arrivals_list_get

In [70]:
import DataFetcher

In [69]:

lines = [123, 202, 921, 970, 148, 55, 82, 17, 18, 138, 70]
TIME_PERIOD_TEST = {
    'start': '2023-01-01T00:00:00+03:00',
    'end': '2023-01-10T00:00:00+03:00' # todo - change this whole year
}

TIME_PERIOD_TRAIN = {
    'start': '2022-01-01T00:00:00+03:00',
    'end': '2022-01-10T00:00:00+03:00' # todo - change this whole year
}

In [ ]:


lines_data = pd.DataFrame()

# Iterate each line and append to the lines_data dataframe
for line in lines:
    new_line_by_short_name = pd.DataFrame(
            stride.iterate('/gtfs_routes/list',   {
                "route_short_name": line,
                "limit": 5
          }, limit = 5)
    )
    
    lines_data = lines_data.append(new_line_by_short_name)

In [ ]:
lines_data

In [ ]:
# lines_data.to_csv("lines_data.csv")

In [ ]:
line_refs_unq = lines_data["line_ref"].unique()
unique_operator_ref = lines_data["operator_ref"].unique()

In [ ]:
siri_line_ids = pd.DataFrame(
    stride.iterate('/siri_routes/list',   {
        "operator_refs": ",".join([str(opr_ref) for opr_ref in unique_operator_ref]),
        "line_refs": ",".join([str(line_ref) for line_ref in line_refs_unq]),
        "limit": 10000
  }, limit = 10000)
)

# join lines data with siri route names
lines_data = lines_data.merge(siri_line_ids, left_on = ["operator_ref", "line_ref"], right_on = ["operator_ref", "line_ref"], suffixes=("_gtfs", "_siri"))

In [ ]:
# Get GTFS Data
gtfs_data_all_stops_per_route = pd.DataFrame(
    stride.iterate('/route_timetable/list',   {
        "planned_start_time_date_from": "2023-08-01T00:00:00+03:00",
        "planned_start_time_date_to": "2023-08-02T00:00:00+03:00",
        "line_refs": ",".join([str(line_ref) for line_ref in line_refs_unq]),
        "limit": 10000
  }, limit = 10000)
)

In [ ]:
gtfs_data_all_stops_per_route

In [ ]:
gtfs_data_all_stops_per_route.to_csv("gtfs_data_all_stops_per_route.csv")

In [ ]:
gtfs_data_all_stops_per_route = pd.read_csv("gtfs_data_all_stops_per_route.csv")

In [ ]:
# Get GTFS Data
siri_data_all_stops_per_route = pd.DataFrame(
    stride.iterate('/siri_rides/list',   {
        "gtfs_ride__start_time_from": "2023-08-01T00:00:00+03:00",
        "gtfs_ride__start_time_to": "2023-08-02T00:00:00+03:00",
        "gtfs_route__line_refs": ",".join([str(line_ref) for line_ref in line_refs_unq]),
        "limit": 10000
  }, limit = 10000)
)

In [68]:
siri_data_all_stops_per_route

,id,siri_route_id,journey_ref,scheduled_start_time,vehicle_ref,updated_first_last_vehicle_locations,first_vehicle_location_id,last_vehicle_location_id,updated_duration_minutes,duration_minutes,...,gtfs_route__date,gtfs_route__line_ref,gtfs_route__operator_ref,gtfs_route__route_short_name,gtfs_route__route_long_name,gtfs_route__route_mkt,gtfs_route__route_direction,gtfs_route__route_alternative,gtfs_route__agency_name,gtfs_route__route_type
0,45074730,2433,2023-07-31-36723838,2023-07-30 21:00:00+00:00,5748287,2023-07-30 21:56:07.784634+00:00,2413029205,2413111021,2023-07-31 04:48:55.650139+00:00,50,...,2023-08-01,10533,18,148,קדושת לוי/שלום רב-ביתר עילית<->יוסף קארו/בן אי...,12148,1,#,קווים,3
1,45074731,2437,2023-07-31-22387075,2023-07-30 21:00:00+00:00,5750787,2023-07-30 21:56:08.050406+00:00,2413029214,2413114627,2023-07-31 04:48:55.918939+00:00,53,...,2023-08-01,10534,18,148,בן איש חי/יוסף קארו-בית שמש<->קדושת לוי/אוהל ש...,12148,2,#,קווים,3
2,45074988,184,2023-07-31-584714129,2023-07-30 21:10:00+00:00,9269501,2023-07-30 21:58:07.363981+00:00,2413047567,2413114844,2023-07-31 04:49:18.796646+00:00,44,...,2023-08-01,2542,5,70,ת.מרכזית תל אביב קומה 7/רציפים-תל אביב יפו<->ב...,20070,1,0,דן,3
3,45075098,666,2023-07-31-2974253,2023-07-30 21:15:00+00:00,9226601,2023-07-31 01:45:10.114716+00:00,2413058522,2413132138,2023-07-31 05:18:20.534830+00:00,55,...,2023-08-01,2328,5,18,ת.רכבת תל אביב - סבידור/רציפים C-תל אביב יפו<-...,22018,1,0,דן,3
4,45075125,6121,2023-07-31-18622316,2023-07-30 21:15:00+00:00,7613869,2023-07-31 01:45:16.985663+00:00,2413059341,2413172517,2023-07-31 05:18:27.775625+00:00,125,...,2023-08-01,6821,3,970,חזון אי''ש/האדמור מנדבורנא-בני ברק<->ערד/סובה-...,10970,1,5,אגד,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,45293471,3290,2023-08-01-9796861,2023-08-01 19:50:00+00:00,30684903,2023-08-01 23:10:59.972846+00:00,2425866740,2425877667,2023-08-02 03:00:41.933398+00:00,5,...,2023-08-01,11016,18,138,ת. רכבת בית שמש-בית שמש<->הר''ן/קדושת לוי-ביתר...,11138,2,#,קווים,3
1115,45293602,4366,2023-08-01-585019478,2023-08-01 20:00:00+00:00,35398902,2023-08-01 23:11:44.203225+00:00,2425869369,2425898728,2023-08-02 03:01:27.862198+00:00,14,...,2023-08-01,11197,18,202,ת. רכבת קרית אריה-פתח תקווה<->שדרות החשמונאים/...,11202,2,0,קווים,3
1116,45293632,2433,2023-08-01-25952386,2023-08-01 19:30:00+00:00,5750087,2023-08-01 23:11:53.788182+00:00,2425870942,2425881789,2023-08-02 03:01:38.420581+00:00,4,...,2023-08-01,10533,18,148,קדושת לוי/שלום רב-ביתר עילית<->יוסף קארו/בן אי...,12148,1,#,קווים,3
1117,45293633,2437,2023-08-01-13422017,2023-08-01 20:00:00+00:00,5595587,2023-08-01 23:11:54.134107+00:00,2425870959,2425920259,2023-08-02 03:01:38.741109+00:00,19,...,2023-08-01,10534,18,148,בן איש חי/יוסף קארו-בית שמש<->קדושת לוי/אוהל ש...,12148,2,#,קווים,3


In [ ]:
siri_ride_ids_unq = siri_data_all_stops_per_route["id"].unique()

In [ ]:
# Get siri routes ids per each of the gtfs ride ids
siri_ride_stops = pd.DataFrame(
    stride.iterate('/siri_ride_stops/list',   {
        "siri_ride_ids": ",".join([str(siri_ride_id) for siri_ride_id in siri_ride_ids_unq[:100]]),
        "siri_ride__scheduled_start_time_from": "2023-08-01T00:00:00+03:00",
        "siri_ride__scheduled_start_time_to": "2023-08-02T00:00:00+03:00",
        "limit": 10000
  }, limit = 10000)
)

# join lines data with siri route names
# lines_data = lines_data.merge(siri_line_ids, left_on = ["operator_ref", "line_ref"], right_on = ["operator_ref", "line_ref"], suffixes=("_gtfs", "_siri"))

In [ ]:
# iterate every 100 siri ride ids
# then join the results since we can't send all together
siri_ride_stops = pd.DataFrame()

for i in range(0, len(siri_ride_ids_unq), 100):
    siri_ride_stops = siri_ride_stops.append(
        pd.DataFrame(
            stride.iterate('/siri_ride_stops/list', {
                "siri_ride_ids": ",".join([str(siri_ride_id) for siri_ride_id in siri_ride_ids_unq[i:i+100]]),
                "siri_ride__scheduled_start_time_from": "2023-08-01T00:00:00+03:00",
                "siri_ride__scheduled_start_time_to": "2023-08-02T00:00:00+03:00",
                "limit": 10000
          }, limit = 10000)
        )
    )

In [64]:
siri_ride_stops

,id,siri_stop_id,siri_ride_id,order,gtfs_stop_id,nearest_siri_vehicle_location_id,siri_ride__siri_route_id,siri_ride__journey_ref,siri_ride__scheduled_start_time,siri_ride__vehicle_ref,...,gtfs_route__operator_ref,gtfs_route__route_short_name,gtfs_route__route_long_name,gtfs_route__route_mkt,gtfs_route__route_direction,gtfs_route__route_alternative,gtfs_route__agency_name,gtfs_route__route_type,scheduled_vs_real_time_difference,scheduled_vs_real_time_difference_seconds
0,1177805520,5755,45188875,80,15785704,2.420355e+09,5481,2023-08-01-25328227,2023-08-01 02:45:00+00:00,7613969,...,3,921,ת.מרכזית תל אביב קומה 7/רציפים-תל אביב יפו<->ת...,10921,1,#,אגד,3,0 days 00:08:15,495.0
1,1177790219,8574,45188875,79,15784356,2.420310e+09,5481,2023-08-01-25328227,2023-08-01 02:45:00+00:00,7613969,...,3,921,ת.מרכזית תל אביב קומה 7/רציפים-תל אביב יפו<->ת...,10921,1,#,אגד,3,0 days 00:02:06,126.0
2,1177784260,5817,45188875,78,15784348,2.420297e+09,5481,2023-08-01-25328227,2023-08-01 02:45:00+00:00,7613969,...,3,921,ת.מרכזית תל אביב קומה 7/רציפים-תל אביב יפו<->ת...,10921,1,#,אגד,3,0 days 00:01:24,84.0
3,1177778039,4092,45188875,76,15785046,2.420285e+09,5481,2023-08-01-25328227,2023-08-01 02:45:00+00:00,7613969,...,3,921,ת.מרכזית תל אביב קומה 7/רציפים-תל אביב יפו<->ת...,10921,1,#,אגד,3,0 days 00:00:53,53.0
4,1177771906,467,45188875,75,15785423,2.420278e+09,5481,2023-08-01-25328227,2023-08-01 02:45:00+00:00,7613969,...,3,921,ת.מרכזית תל אביב קומה 7/רציפים-תל אביב יפו<->ת...,10921,1,#,אגד,3,0 days 00:00:03,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,1180213723,995,45291412,2,15792445,2.425767e+09,666,2023-08-01-585056931,2023-08-01 19:53:00+00:00,9304601,...,5,18,ת.רכבת תל אביב - סבידור/רציפים C-תל אביב יפו<-...,22018,1,0,דן,3,0 days 00:01:59,119.0
339,1180212900,1323,45291883,1,15811087,2.425765e+09,1853,2023-08-01-1327544,2023-08-01 20:00:00+00:00,7825369,...,3,123,ת. מרכזית חוף הכרמל/רציפים עירוני-חיפה<->ת. מר...,10123,1,#,אגד,3,NaT,NaN
340,1180212897,365,45291881,1,15810983,2.425781e+09,1851,2023-08-01-1327993,2023-08-01 20:00:00+00:00,23313802,...,3,123,ת. מרכזית המפרץ/רציפים עירוני-חיפה<->ת. מרכזית...,10123,2,#,אגד,3,NaT,NaN
341,1180212325,471,45291760,1,15799002,2.425770e+09,4366,2023-08-01-0,2023-08-01 19:56:00+00:00,35398902,...,18,202,ת. רכבת קרית אריה-פתח תקווה<->שדרות החשמונאים/...,11202,2,0,קווים,3,-1 days +23:54:03,-357.0


In [ ]:
siri_ride_stops.to_csv("siri_ride_stops.csv")

In [ ]:
siri_ride_stops["scheduled_vs_real_time_difference"] = siri_ride_stops["nearest_siri_vehicle_location__recorded_at_time"] - siri_ride_stops["gtfs_ride_stop__arrival_time"]
siri_ride_stops["scheduled_vs_real_time_difference_seconds"] = siri_ride_stops["scheduled_vs_real_time_difference"].dt.total_seconds()

In [ ]:
siri_ride_stops[["nearest_siri_vehicle_location__recorded_at_time", "gtfs_ride_stop__arrival_time", "scheduled_vs_real_time_difference", "scheduled_vs_real_time_difference_seconds"]]

In [ ]:
# plot time difference
px.histogram(siri_ride_stops, x = "scheduled_vs_real_time_difference_seconds", nbins = 100)

In [ ]:
px.histogram(siri_ride_stops, x = "nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters", nbins = 100)

In [ ]:
siri_ride_stops[siri_ride_stops['nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters'] > 10000]

In [ ]:
no_outliers = siri_ride_stops[siri_ride_stops['nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters'] < 1000]
no_outliers = no_outliers[no_outliers['scheduled_vs_real_time_difference_seconds'] < 3600]

In [ ]:
# See correlation between time difference and distance from stop
px.box(no_outliers, x = "gtfs_ride_stop__stop_sequence", y = "scheduled_vs_real_time_difference_seconds")

In [67]:
# px.box(no_outliers, x = "gtfs_stop__city", y = "scheduled_vs_real_time_difference_seconds")

In [ ]:
# gtfs_data["gtfs_line_ref"] == '19713'

In [ ]:
# This shows all of the stops and their planned arrival time
gtfs_data[gtfs_data["gtfs_line_ref"] == '19713']

In [ ]:
#FINDING THE INFORAMTION ABOUT SIRI VEHICKE 39344
#RES:
#   "id": 39344,
#   "siri_snapshot_id": 8,
#   "siri_ride_stop_id": 16937,
#   "recorded_at_time": "2022-03-09T11:13:07+00:00",
#   "lon": 34.887203,
#   "lat": 32.091144,
#   "bearing": 230,
#   "velocity": 14,
#   "distance_from_journey_start": 3254,
#   "distance_from_siri_ride_stop_meters": 66

#SIRI SNAPSHOT RES:


# gtfs_data explanation:
* the lon stands for longitude
* the lat stands for latitude.
* both of them are coordinates of the bus stop.
* the planned arrival time is the time the bus is planned to arrive at the stop.
* the gtfs_lin_ref is id of the bus trip, meaning each bus line direction has its one id.
* for example 19717 is line 202 by Eged, driving from Hifa to Pardes Hana, where 19716 is the opposite way.
* the gtfs_line_start_time is the time the bus is planned to start the trip from its origin.
* the gtfs_ride_id is the specific id of the bus trip, meaning each bus trip has its own id.


In [ ]:
# Get the start and end of each ride, we are only interested in full rides at this stage
lines_start_end = gtfs_data.groupby(["gtfs_ride_id", "gtfs_line_ref"]).agg(
    start = ("gtfs_line_start_time", "min"),
    end = ("planned_arrival_time", "max")
).reset_index()

In [ ]:
# Calculate the duration
lines_start_end["duration"] = lines_start_end["end"] - lines_start_end["start"]
lines_start_end["duration_minutes"] = lines_start_end["duration"].dt.total_seconds() / 60

As we can see, there is only a single duration for each ride.

That is because the rides are planned and not real time.

In [ ]:
lines_start_end.groupby("gtfs_line_ref").agg(
    distinct_durations = ("duration_minutes", "nunique"),
    duration_example = ("duration_minutes", "first")
)

In [ ]:
lines_start_end.agg(
    num_of_rides = ("gtfs_ride_id", "count"),
    min_start = ("start", "min"),
    max_end = ("end", "max")
)

# Get real time data (SIRI)

In [ ]:
siri_routes = stride.get('/siri_routes/list', {'operator_refs': '3,31', 'line_refs': '19713,19717,1806'})
pd.DataFrame(siri_routes)


In [ ]:
# Get the siri route ids seperated by commas 
siri_route_ids = ','.join([str(route['id']) for route in siri_routes])
siri_route_ids

### Get rides data

Here the data is already aggregated per ride, and we get the real time data for each ride.

In [ ]:
# get the data of only Merch 2023, 2022 and 2021
df_2023 = pd.DataFrame(stride.iterate('/siri_rides/list', {
    'siri_route_ids': siri_route_ids, 
    'scheduled_start_time_from': "2023-03-01T00:00:00+03:00",
    'scheduled_start_time_to': "2023-03-31T23:59:59+03:00",
    'order_by': 'scheduled_start_time desc',
    'limit': 50000
}, limit=20000))
df_2022 = pd.DataFrame(stride.iterate('/siri_rides/list', {
    'siri_route_ids': siri_route_ids, 
    'scheduled_start_time_from': "2022-03-01T00:00:00+03:00",
    'scheduled_start_time_to': "2022-03-31T23:59:59+03:00",
    'order_by': 'scheduled_start_time desc',
    'limit': 50000
}, limit=20000))
march_df = pd.concat([df_2023, df_2022])

In [ ]:

df = pd.DataFrame(stride.iterate('/siri_rides/list', {
    'siri_route_ids': siri_route_ids, 
    'scheduled_start_time_from': "2023-08-01T00:00:00+03:00",
    'order_by': 'scheduled_start_time desc',
    'limit': 50000
}, limit=20000))
df

# Data exploration
siri_route_id: The related siri_route.
journey_ref": "A unique identifier for this ride as provided by the SIRI data.
scheduled_start_time: The scheduled start time as provided by the SIRI data. Note that this value may change over time but we only show the first value as received from the SIRI real-time data.
vehicle_ref: A unique identifier of the bus or vehicle. This may be the license number but could also be other identifier as provided by the SIRI data.
updated_first_last_vehicle_locations: The time the bus was last seen in the system.
first_vehicle_location_id: A unique identifier of the bus or vehicle. This may be the license number but could also be other identifier as provided by the SIRI data.
last_vehicle_location_id: The last siri_vehicle_location along this ride.
updated_duration_minutes: The updated duration of this ride in minutes
duration_minutes: The duration of this ride in minutes
journey_gtfs_ride_id: The related gtfs_ride based on journey_ref.
route_gtfs_ride_id: The related gtfs_ride based on operator_ref, line_ref and scheduled_start_time.
gtfs_ride_id: The related gtfs_ride based on best match from either journey_gtfs_ride_id or route_gtfs_ride_id. 
siri_route__line_ref: Unique identifier for this route, together with operator_ref.
siri_route__operator_ref: Unique identifier for this route, together with line_ref.
gtfs_ride__gtfs_route_id: The related gtfs_route this ride is a part of.
gtfs_ride__journey_ref: A unique identifier for this ride as provided by the original MOT GTFS data.
gtfs_ride__start_time: The start time of this ride.
gtfs_ride__end_time: The end time of this ride.
gtfs_route__date: The date of this ride.
gtfs_route__line_ref: Unique identifier for this route, together with operator_ref.
gtfs_route__operator_ref: Unique identifier for this route, together with line_ref.
gtfs_route__route_short_name: The route short name as provided by the original MOT GTFS data.
gtfs_route__route_long_name: The route long name as provided by the original MOT GTFS data.
gtfs_route__route_mkt: The route mkt as provided by the original MOT GTFS data.
gtfs_route__route_direction: The route direction as provided by the original MOT GTFS data.
gtfs_route__route_alternative: The route alternative as provided by the original MOT GTFS data.
gtfs_route__agency_name: The agency name as provided by the original MOT GTFS data.
gtfs_route__route_type: The route type as provided by the original MOT GTFS data.

# Initial EDA and ideation

### Look for outliers, 

Clean the nulls in the real-time data

Look for clear outliers in the data

In [ ]:
df = df.drop_duplicates()

In [ ]:
# print the df where the duration minutes is null
df[df.duration_minutes.isnull()]

Get the mean duration for all those rides

In [ ]:
# Get the mean duration for all the rides
possible_outliers = df.groupby("gtfs_route__line_ref").agg(
    mean_duration = ("duration_minutes", "mean"),
    std_duration = ("duration_minutes", "std"),
    median_duration = ("duration_minutes", "median"),
    distinct_durations = ("duration_minutes", "nunique"),
    duration_example = ("duration_minutes", "first")
).reset_index()

possible_outliers

In [ ]:
march_possible_outliers = march_df.groupby("gtfs_route__line_ref").agg(
    mean_duration = ("duration_minutes", "mean"),
    std_duration = ("duration_minutes", "std"),
    median_duration = ("duration_minutes", "median"),
    distinct_durations = ("duration_minutes", "nunique"),
    duration_example = ("duration_minutes", "first")
).reset_index()

march_possible_outliers

### Outliers

We're seeing some outliers in the data, see the dots that are near zero - they make no sense.

There are dots as well that are way above the mean, we will need to investigate those as well.

Current thought is to remove any ride that is more then 2 standard deviations from the mean. 

In [ ]:
px.scatter(df, x = "scheduled_start_time", y = "duration_minutes", color = "gtfs_route__line_ref")

In [ ]:
px.scatter(march_df, x = "scheduled_start_time", y = "duration_minutes", color = "gtfs_route__line_ref")

In [ ]:
# Join the df with the outliers data based on the gtf_route_line_ref column

all_rides = df.merge(possible_outliers, left_on="gtfs_route__line_ref", right_on="gtfs_route__line_ref")
march_all_rides = march_df.merge(march_possible_outliers, left_on="gtfs_route__line_ref", right_on="gtfs_route__line_ref")

In [ ]:
all_rides_filtered_outliers = all_rides[(all_rides.duration_minutes > (all_rides.mean_duration - 2 * all_rides.std_duration)) &
          (all_rides.duration_minutes < (all_rides.mean_duration + 2 * all_rides.std_duration))]
march_all_rides_filtered_outliers = march_all_rides[(march_all_rides.duration_minutes > (march_all_rides.mean_duration - 2 * march_all_rides.std_duration)) &
          (march_all_rides.duration_minutes < (march_all_rides.mean_duration + 2 * march_all_rides.std_duration))]

### Outliers removed

This looks much better!

In [ ]:
px.scatter(all_rides_filtered_outliers, x = "scheduled_start_time", y = "duration_minutes", color = "gtfs_route__line_ref")

In [ ]:
px.scatter(march_all_rides_filtered_outliers, x = "scheduled_start_time", y = "duration_minutes", color = "gtfs_route__line_ref")

In [ ]:
lines_start_end["gtfs_ride_id"] = lines_start_end["gtfs_ride_id"].astype(int)
all_rides_filtered_outliers["gtfs_ride_id"] = all_rides_filtered_outliers["gtfs_ride_id"].astype(int)
march_all_rides_filtered_outliers["gtfs_ride_id"] = march_all_rides_filtered_outliers["gtfs_ride_id"].astype(int)

In [ ]:
# Join planned data with actual data
all_data_joined = all_rides_filtered_outliers.merge(lines_start_end, on = "gtfs_ride_id", how = "inner", suffixes=("_actual", "_planned"))
march_all_data_joined = march_all_rides_filtered_outliers.merge(lines_start_end, on = "gtfs_ride_id", how = "inner", suffixes=("_actual", "_planned"))
all_data_joined

In [ ]:
lines_start_end

In [ ]:
all_data_joined["real_time_difference"] = all_data_joined["duration_minutes_actual"] - all_data_joined["duration_minutes_planned"]
march_all_data_joined["real_time_difference"] = march_all_data_joined["duration_minutes_actual"] - march_all_data_joined["duration_minutes_planned"]

### Time difference

Here we have a clear view on the time difference between the planned and actual ride.

We see there are almost no rides that are shorter then planned, and most of the rides are longer then planned, which makes sense.

By a quick glance, we can see that most of the rides are late by around ~20 minutes.

In [ ]:
px.histogram(all_data_joined, x = "real_time_difference", color = "gtfs_route__line_ref", nbins = 100, barmode="group")

In [ ]:
px.histogram(march_all_data_joined, x = "real_time_difference", color = "gtfs_route__line_ref", nbins = 100, barmode="group")

### Not all rides are the same

20 minutes for a short bus is not the same as 20 minutes for a long bus.

In the chart below, we showcase on much time was added in relation to the planned ride duration.

We can see that for the line in Pardes Hanna (19717) the ride usually gets longer by almost 40% !!! 

In [ ]:
# Average time differences in relation to the total planned trip duration
ride_stats_avg = all_data_joined.groupby("gtfs_route__line_ref").agg(
    mean_real_time_difference = ("real_time_difference", "mean"),
    mode_planned_duration = ("duration_minutes_planned", "median")
)

ride_stats_avg["ratio_time_added_to_planned_ride"] = (ride_stats_avg["mean_real_time_difference"] / ride_stats_avg["mode_planned_duration"]) * 100
ride_stats_avg

On average, regardless of the bus, the difference is arond 20 minutes.

In [ ]:
# Total mean mean real time difference
total_mean_time_difference = all_data_joined["real_time_difference"].mean()
total_mean_time_difference

# Look for trends in hours and days

The idea is simple - the bus is more prone to be late during rush hours, and less prone to be late during the night.

Also, the bus will probably be more busy in Sunday than in a regular Tuesday, so it will be more prone to be late.

We will check if this is true.

In [ ]:
all_data_joined["day_of_week"] = all_data_joined["scheduled_start_time"].dt.day_name()
all_data_joined["hour_of_day"] = all_data_joined["scheduled_start_time"].dt.hour

The following plot looks a bit messy, as we can't really understand a clear trend.

Lets try and group that by the mean of the real time difference, see if we can see the trend there.

In [ ]:
px.scatter(all_data_joined, x = "hour_of_day", y = "real_time_difference")

Much better!

There is a clear rise around 1 PM, and a clear drop around 8 PM.

In [ ]:
to_plot_hours = all_data_joined.groupby("hour_of_day").agg(
    mean_real_time_difference = ("real_time_difference", "mean")
).reset_index()

px.line(to_plot_hours, x = "hour_of_day", y = "mean_real_time_difference")

In [ ]:
to_plot_hours = all_data_joined.groupby("day_of_week").agg(
    mean_real_time_difference = ("real_time_difference", "mean")
).reset_index()

px.line(to_plot_hours, x = "day_of_week", y = "mean_real_time_difference")

In [ ]:
to_plot_hours = all_data_joined.groupby(["day_of_week", "hour_of_day"]).agg(
    mean_real_time_difference = ("real_time_difference", "mean")
).reset_index()

px.line(to_plot_hours, x = "hour_of_day", y = "mean_real_time_difference", color = "day_of_week")


# Model predictions

In this chapter we try and predict the real time delay, as a way of predicting how long the trip will actually take.

We could think of numerous ways to do that, but we will start with a simple linear regression model.

Later on we'll showcase a time series based model (ARIMA).


### Linear regression

We will use the following features:

- Planned trip duration
- Hour of day
- Day of week

We will try and predict the real time difference, and see how well we do.


In [ ]:
# Try to predict the real time difference with linear regression

# Split all data joined by date

test = all_data_joined[all_data_joined["scheduled_start_time"] > "2023-09-01"]
train = all_data_joined[all_data_joined["scheduled_start_time"] <= "2023-09-01"]

print("amount of train data: ", len(train), "amount of test data: ", len(test))

sample_data_train = test[["duration_minutes_planned", "hour_of_day", "day_of_week", "real_time_difference"]]
sample_data_test = train[["duration_minutes_planned", "hour_of_day", "day_of_week", "real_time_difference"]]
# Convert day of week to number, first day is sunday
sample_data_train["day_of_week"] = sample_data_train["day_of_week"].map({
    "Sunday": 1,
    "Monday": 2,
    "Tuesday": 3,
    "Wednesday": 4,
    "Thursday": 5,
    "Friday": 6,
    "Saturday": 7
})

sample_data_test["day_of_week"] = sample_data_test["day_of_week"].map({
    "Sunday": 1,
    "Monday": 2,
    "Tuesday": 3,
    "Wednesday": 4,
    "Thursday": 5,
    "Friday": 6,
    "Saturday": 7
})

X_train = sample_data_train[["duration_minutes_planned", "hour_of_day", "day_of_week"]]
Y_train = sample_data_train["real_time_difference"]
X_test = sample_data_test[["duration_minutes_planned", "hour_of_day", "day_of_week"]]
y_test = sample_data_test["real_time_difference"]

In [ ]:
# Fit the model
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
# Get the predictions

y_pred = model.predict(X_test)

In [ ]:
# Plot the predictions
px.scatter(x = y_test, y = y_pred)

In [ ]:
# calculate presicion

from sklearn.metrics import mean_squared_error, r2_score

print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))

# The coefficient of determination:
print('Variance: %.2f' % r2_score(y_test, y_pred))

## Linear regression results

The results for the model are not great, but not bad either.

We can see that on average we're still missing around 13 minutes.

Compared to the average delay of 20 minutes, it's not bad.

In [ ]:
math.sqrt(mean_squared_error(y_test, y_pred))

### Diving deeper

Looking into the differences, we can see we are highly affected by some outliers.

It might be worth further investigating those outliers, and see if we can find a way to improve the model.

In [ ]:
# Calculate the abs diff between y_test and y_pred

y_test_pred = pd.DataFrame({"y_test": y_test, "y_pred": y_pred})
y_test_pred["abs_diff"] = abs(y_test_pred["y_test"] - y_test_pred["y_pred"])
px.histogram(y_test_pred, x = "abs_diff", nbins = 100)

## ARIMA Model

ARIMA Is a time series model 

https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average

It's usually being used to predict stock prices, but we can use it to predict the real time difference.

This is a simple model that can showcase what it looks like to use time series models.

This models takes no features but the time series itself, and tries to predict the next value based on the previous values.



In [ ]:
!pip install statsmodels
from statsmodels.tsa.arima.model import ARIMA


### Preparing the data

In order for the results and the analysis to be clear, we filtered this model for only a single line.

This way there will be no confusion about the results, and there will only be a single value per time.

In [ ]:
# Filter for a single line only, for now
all_data_joined_single_line = all_data_joined[all_data_joined["gtfs_route__line_ref"] == 19713]
data_for_time_series = all_data_joined_single_line[["scheduled_start_time", "real_time_difference"]]

train = data_for_time_series[data_for_time_series["scheduled_start_time"] < "2023-09-01"]
test = data_for_time_series[data_for_time_series["scheduled_start_time"] > "2023-09-01"]

In [ ]:
print("amount of train data: ", len(train), "amount of test data: ", len(test))

In [ ]:
y = train["real_time_difference"]

In [ ]:
ARIMAmodel = ARIMA(y, order = (2, 2, 2))
ARIMAmodel = ARIMAmodel.fit()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
y_pred = ARIMAmodel.get_forecast(len(test.scheduled_start_time))
y_pred_df = y_pred.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.scheduled_start_time
y_pred_out = y_pred_df["Predictions"] 
# Plot by each line
plt.plot(y_pred_out, color='Yellow', label = 'ARIMA Predictions')
plt.legend()


import numpy as np
from sklearn.metrics import mean_squared_error

arma_rmse = np.sqrt(mean_squared_error(test["real_time_difference"].values, y_pred_df["Predictions"]))
print("RMSE: ",arma_rmse)

### ARIMA results

We can see that the model almost always predict a delay that's between 22 and 23 minutes.

This does not seem good and does not correlate with real world data.

In the sqrt of the mean squared error, we're getting a far worse result than the linear regression model, 16.5 compared to 13, but that's still better than counting on the non-real time data (average of 20 minutes difference).

In [ ]:
px.scatter(x = test["real_time_difference"].values, y = y_pred_df["Predictions"])

# Model comparison and conclusions

Overall, the linear regression model seems to be better than the ARIMA model.

The linear regression model is more accurate, and it's easier to understand and explain.

The ARIMA model is a time series model, and it's not as easy to understand and explain.

The ARIMA model is also not as flexible as the linear regression model, and it's harder to add features to it.

We think that some of the differences relay on the big difference between August and September, making it difficult for the model's predictions.


# Next Steps

1. Use a more advanced model for time series that is fitted for similar data.
2. Use more features to improve the linear regression model.
3. Add more feature-based models to compare with the linear regression model.
4. Try to find a way to improve the data quality, reducing outliers.
5. Add more data - more lines and more time.
6. Add a deep neural network model with memory such as LSTM.
7. Use additional data sources such as weather data.